# Brightway importer for environmentally extended Input-Output tables

**Goal**: Create an Brightway importer for the Bonsai database. The database is the output of the [_Getting the Data Right_](https://vbn.aau.dk/en/projects/getting-the-data-right) project. 

some words about Bonsai:

- The entire workflow to create the database is **open-source**, with the aim to foster collaboration in the co-creation of open databases for environmental footprinting. The database itself is also open
- At this stage if focuses on **Greenhouse gas emissions** but additional _externalities_ have already been added.
- The database follows **consequential** paradigm.

### Approach

- Based in the IOTable backend of bw2data (no need to create a gigantic SQL database if there's no metadata on the edges)
- Inspired in the existing Exiobase importers
- We started with a simple example and therefore created a _generic_ importer that could be used with other EEIO tables (e.g. those coming from [pymrio](https://github.com/IndEcol/pymrio))
- The _final demand_ (i.e. societal demand of products) is integrated in the IO table

In [15]:
import json
from pathlib import Path

import pandas as pd
import numpy as np
import bw2data as bd
import bw2calc as bc
import bw2io
from bw2io.importers.io import IOImporter
from bw2io.strategies.io import tidy_tables

In [16]:
print(bd.__version__)
print(bw2io.__version__)

(4, 0, 'dev54')
0.9.DEV38


In [17]:
bd.projects.set_current('test_io_importer')

In [18]:
bw2io.remote.install_project('ecoinvent-3.10-biosphere',
                             'test_io_importer',
                             overwrite_existing=True)

Restoring project backup archive - this could take a few minutes...
Applying automatic update: 4.0 database search directories
Reindexing database ecoinvent-3.10-biosphere
Restored project: test_io_importer


'test_io_importer'

## define example database

### tidy tables

In [19]:
pet_hiot = pd.DataFrame([[1,-2],[0,1]], # experiment setting 1 prod to 0
             index=pd.MultiIndex.from_tuples([('DK','prod1'),('DK','prod2')]),
             columns = pd.MultiIndex.from_tuples([('DK','act1'),('DK','act2')]))

pet_hiot = pet_hiot.astype(pd.SparseDtype("float",0))


B = pd.DataFrame([[1,3],[1,2],[0,1],[4,0]],
             index=pd.Index(['co2_air','ch4_air','co2_accelerated_air','land_occupation']),
             columns = pd.MultiIndex.from_tuples([('DK','act1'),('DK','act2')]))

B = B.astype(pd.SparseDtype("float",0))

fd = pd.DataFrame.from_dict(
    {
    ('DK','Household'):{('DK','prod1'):-11,('DK','prod2'):-3},
    ('DK','Government'):{('DK','prod1'):-8,('DK','prod2'):-4},
    ('DK','Capital'):{('DK','prod1'):-4,('DK','prod2'):-2}
        }
    )

Bfd = pd.DataFrame([[1,3],],
index=pd.Index(['co2_air',]),
columns = pd.MultiIndex.from_tuples([('DK','Household'),
('DK','Government')])).astype(pd.SparseDtype("float",0))

In [20]:
print(pet_hiot)
print(B)
print(fd)
print(Bfd)

           DK     
         act1 act2
DK prod1  1.0 -2.0
   prod2    0  1.0
                      DK     
                    act1 act2
co2_air              1.0  3.0
ch4_air              1.0  2.0
co2_accelerated_air    0  1.0
land_occupation      4.0    0
                DK                   
         Household Government Capital
DK prod1       -11         -8      -4
   prod2        -3         -4      -2
               DK           
        Household Government
co2_air       1.0        3.0


In [21]:
pfd = pd.DataFrame((np.eye(fd.shape[1])),index=fd.columns,columns=fd.columns)
fd_total = pd.concat([fd,pfd])
fd_total = fd_total.astype(pd.SparseDtype("float",0))

In [22]:
extended_hiot = pd.concat([pet_hiot,fd_total],axis=1).fillna(0)
extended_B = pd.concat([B,Bfd],axis=1).fillna(0)

In [23]:
print(extended_hiot)

                DK                                  
              act1 act2 Household Government Capital
DK prod1       1.0 -2.0     -11.0       -8.0    -4.0
   prod2         0  1.0      -3.0       -4.0    -2.0
   Household   0.0  0.0       1.0          0       0
   Government  0.0  0.0         0        1.0       0
   Capital     0.0  0.0         0          0     1.0


In [24]:
print(extended_B)

                      DK                          
                    act1 act2 Household Government
co2_air              1.0  3.0       1.0        3.0
ch4_air              1.0  2.0       0.0        0.0
co2_accelerated_air    0  1.0       0.0        0.0
land_occupation      4.0    0       0.0        0.0


we have one technosphere and one biosphere matrix. We added some functions to store this as tidy tables in a compressed csv.

In [25]:
path_to_intermediate = (Path.cwd()/'example_data')
path_to_intermediate.mkdir(exist_ok=True)

Add metadata

- units
- names
- _context_ / compartment for elementary flows

In [26]:
metadata_dict = {'prod1':{'unit':'kg','name':'product 1'},
                 'prod2':{'unit':'kg','name':'product 2'},
                 'Household':{'unit':'unit','name':'the household'},
                 'Government':{'unit':'unit','name':'the government'},
                 'Capital':{'unit':'unit','name':'capital investments'},

                 'co2_air':{'unit':'ton', # not standard units
                            'name':'carbon dioxide',
                            'compartment':('air',)},
                 'ch4_air':{'unit':'kg',
                            'name':'methane',
                            'compartment':('air',)},
                 'co2_accelerated_air':{'unit':'kg', # additional biosphere flow
                                        'name':'carbon dioxide accelerated',
                                        'compartment':('air',)},
                 'land_occupation':{'unit':'hectare * year', # non standard composite unit
                                    'name':'land occupation',
                                    'compartment':('natural resource', 'land')}
                }

with open(path_to_intermediate/'io_metadata.json', 'w') as fp:
    json.dump(metadata_dict, fp,indent=4)

In [27]:
tidy_tables(extended_hiot,extended_B,path_to_intermediate)

### Import the database

In [28]:
try:
    del bd.databases['pet_io_db biosphere']
    del bd.databases['pet_io_db']
except KeyError:
    print('db not there')

db not there


mapping between the elementary flows in the database and those in our _biosphere_ database

These include the links between the database being imported and the biosphere3 database. 

For those that do not have a correspondence, a new elementary flow will be created in an additional biosphere db

In [29]:
b3 = bd.Database('ecoinvent-3.10-biosphere')
co2 = b3.get(name='Carbon dioxide, fossil',categories=('air',))
ch4 = b3.get(name='Methane, fossil',categories=('air',))
land_occupation = b3.get(name='Occupation, unspecified')

In [30]:
biosphere_mapping = {
    'co2_air':co2['code'],
    'ch4_air':ch4['code'],
    'land_occupation':land_occupation['code'],
    }

In [31]:
pet_example = IOImporter(dirpath=path_to_intermediate,
                         db_name='pet_io_db',
                         b3mapping=biosphere_mapping)
pet_example.apply_strategies()
pet_example.write_database()

### test

In [33]:
test_db = bd.Database('pet_io_db')

In [34]:
act1 = test_db.get(code='act1|DK')
act2 = test_db.get(code='act2|DK')

In [35]:
# it should have CO2 emissions close to 1 ton and land use of 40000 m2 * year
for e in act1.biosphere():
    print(e)

Exchange: 40000.0 square meter-year 'Occupation, unspecified' (square meter-year, None, ('natural resource', 'land')) to 'act1' (kilogram, DK, None)>
Exchange: 907.1847534179688 kilogram 'Carbon dioxide, fossil' (kilogram, None, ('air',)) to 'act1' (kilogram, DK, None)>
Exchange: 1.0 kilogram 'Methane, fossil' (kilogram, None, ('air',)) to 'act1' (kilogram, DK, None)>


In [36]:
ipcc_2021 = ('IPCC 2021', 'climate change', 'global warming potential (GWP100)')

In [37]:
test_lca = bc.LCA({act2:1},ipcc_2021)

In [38]:
test_lca.lci()
test_lca.lcia()
test_lca.score

4655.12370300293

In [39]:
extended_hiot

DK                                  
              act1 act2 Household Government Capital
DK prod1       1.0 -2.0     -11.0       -8.0    -4.0
   prod2         0  1.0      -3.0       -4.0    -2.0
   Household   0.0  0.0       1.0          0       0
   Government  0.0  0.0         0        1.0       0
   Capital     0.0  0.0         0          0     1.0

In [40]:
test_lca.technosphere_matrix.todense()

matrix([[  1.,  -2., -11.,  -8.,  -4.],
        [  0.,   1.,  -3.,  -4.,  -2.],
        [  0.,   0.,   1.,   0.,   0.],
        [  0.,   0.,   0.,   1.,   0.],
        [  0.,   0.,   0.,   0.,   1.]])

In [41]:
pd.DataFrame(test_lca.biosphere_matrix.todense())

,0,1,2,3,4
0,40000.000000,0.000000,0.000000,0.000000,0.0
1,907.184753,2721.554199,907.184753,2721.554199,0.0
2,1.000000,2.000000,0.000000,0.000000,0.0
3,0.000000,1.000000,0.000000,0.000000,0.0


it can:

- handle unit conversion (pint)
- handle elementary flows not present in biosphere 
- handle some non-standard composite units (e.g hectare * year)

## Bonsai 

### tidy tables

We've already done this for you :)

### import the database

In [42]:
from bw2io.importers.bonsai import BonsaiImporter
from bw2io.strategies.bonsai import mapb3

In [43]:
bonsai_data_path = Path('bonsai_data')

In [44]:
importer = BonsaiImporter(dirpath=bonsai_data_path,
                          db_name='bonsai',
                          b3mapping=mapb3(),
                          )

In [45]:
importer.apply_strategies()

In [46]:
importer.write_database()

100%|██████████| 2/2 [00:00<00:00, 5165.40it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 34567/34567 [00:06<00:00, 5550.46it/s]


Vacuuming database 
Starting IO table write
Adding technosphere matrix
Adding biosphere matrix
Finalizing serialization


### test bonsai

In [49]:
bonsai_db = bd.Database('bonsai')

In [53]:
locations = {a['location'] for a in bonsai_db}
names = {a['name'] for a in bonsai_db}

In [56]:
print(f"{len(locations)} locations, {len(names)} different products")

50 locations, 783 different products


In [63]:
len(bonsai_db)

34567

In [58]:
act = bonsai_db.random()
act

In [59]:
# without pypardiso this is painfully slow
lca = act.lca(ipcc_2021)

In [62]:
density = lca.technosphere_matrix.getnnz() / np.prod(lca.technosphere_matrix.shape)
print(f"technosphere density of {density:.2%}")

technosphere density of 0.12%


In [67]:
lca.to_dataframe().pivot_table(index='row_name',aggfunc='sum',values='amount')

,amount
row_name,
"Carbon dioxide, fossil",405.410048
Dinitrogen monoxide,239.487907
"Methane, fossil",27.984032
"Methane, non-fossil",2.138015


In [71]:
lca.to_dataframe().pivot_table(index='col_name',
                               aggfunc='sum',
                               values='amount').sort_values(by='amount',
                                                            ascending=False).head(10)

,amount
col_name,
heat from combined heat and power,136.083840
"land transformation, intensification of cropland",109.872252
sunflower seed,104.751225
"Gas/Diesel Oil, combustion of",86.757773
"Natural gas and services related to natural gas extraction, excluding surveying, combustion of",62.809355
heat (steam and hot water) from heat production,45.376637
electricity by coal,30.421915
"Motor Gasoline, combustion of",16.872697
crude petroleum extraction,14.974992
